In [4]:
folder_path = '../Data/multilexnorm/data/'
file_path = 'en/train.norm'

In [5]:
import pandas as pd

In [11]:
df = pd.read_csv(folder_path + file_path, sep='\t', quotechar="'")
df.head(11)

,rt,rt.1
0,@teddyferrari1,@teddyferrari1
1,:,:
2,"""",""""
3,ah,ah
4,...,...
5,@datzmenoni,@datzmenoni
6,:,:
7,why,why
8,sub,sub
9,ozil,ozil
